In [ ]:
# 1. Content-based recommender 
# 2. Collaborative-based recommender 
# Classificação: Parallel - Weighted (Heterogenous data type) 
# dataset_1: https://grouplens.org/datasets/movielens/100k/ 
# dataset_2: https://www.kaggle.com/rounakbanik/the-movies-dataset/data#movies_meta data.csv

In [1]:
import pandas as pd 
import numpy as np

### Selecionando e separando as Avaliações

In [8]:
# Usado para o Collaborative-recommender, e para filtrar os filmes a serem classificados 
# ao todo temos 100.000 avaliações de 943 usuários em 1682 filmes 
# cada usuário avaliou pelo menos 20 filmes 
ratingsColumn = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('u.data', sep='\t', names=ratingsColumn, encoding='latin-1')

# Removendo a Coluna timestamp
ratings = ratings.drop('timestamp', axis=1)
ratings.head(5)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Trabalhando com o segundo dataset, filtrando os filmes que tem alguma avaliação

In [9]:
overview = pd.read_csv("movies_metadata.csv", low_memory=False)
# selecionando os campos mais 'relevantes' 
overview = overview[['id','title', 'overview', 'vote_count','vote_average']]
overview.head(5)

,id,title,overview,vote_count,vote_average
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",5415.0,7.7
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,2413.0,6.9
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,92.0,6.5
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",34.0,6.1
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,173.0,5.7


In [12]:
# Criamos ums lista de todos os ids dos filmes que possuem avaliação 
movies_rated_index = ratings["movie_id"].tolist()

# Convertendo esse valor para string
movies_rated_index = [str(i) for i in movies_rated_index]

# Retonamos uma cópia do dataset, somente com os filmes que possuem avaliação 
# Fazendo um Join nos dois datasets para ter o overview apenas dos filmes que tem avaliações
overview = overview.copy().loc[overview['id'].isin(movies_rated_index)] 
overview.shape 
# agora temos 1068 filmes no dataset


(1068, 5)

### Criando matriz com frequência de palavras *(Biblioteca Externa)* 

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""
stopwords Definition
Stop words are words like “and”, “the”, “him”, which are presumed to be uninformative 
in representing the content of a text, and which may be removed to avoid them being construed 
as signal for prediction. Sometimes, however, similar words are useful for prediction, such 
as in classifying writing style or personality.

"""
tfidf = TfidfVectorizer(stop_words='english')
    
# Constrói a matriz de frequência de palavras
# Learn vocabulary and idf, return term-document matrix.

# A document-term matrix or term-document matrix is a mathematical matrix that describes the frequency of 
# terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents 
# in the collection and columns correspond to terms. There are various schemes for determining the value 
# that each entry in the matrix should take. One such scheme is tf-idf. They are useful in the field of 
# natural language processing.

tfidf_matrix = tfidf.fit_transform(overview['overview']) 

# Temos 9422 palavras que descrevem 1068 filmes 
# Linhas = filmes 
# Colunas = palavras 
# Então temos a quantidade de cada palavras, em cada overview de cada filme 
# Saída: tfidf_matrix.__class__.__name__ = 'csr_matrix'

tfidf_matrix.shape

(1068, 9422)

### Criando matriz de similaridade *(Biblioteca Externa)* 


In [18]:
from sklearn.metrics.pairwise import linear_kernel 

# Calculando matriz de similaridade
# Compute the linear kernel between X and Y.
# Mais sobre linear kernel https://scikit-learn.org/stable/modules/metrics.html#linear-kernel
# http://crsouza.com/2010/03/17/kernel-functions-for-machine-learning-applications/#linear
"""
Uma matriz de similaridade é uma matriz de pontuação (escores) que expressa a similaridade entre dois pontos dados. 
Matrizes de similaridade estão fortemente relacionadas com os seus homólogos, matrizes de distâncias e matrizes de 
substituição. Uma matriz de similaridade é o conceito oposto ao da matriz de distâncias. Os elementos de uma matriz 
de similaridade medem as semelhanças entre pares de objetos - quanto maior similaridade de dois objetos, maior o 
valor da medida.

"""
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
print(cosine_sim)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


### Determinando indice para cada filme


In [22]:
# Pegando todos os filmes e separando o título e relacionando com o seu id

# Construct a reverse mapping of indices and movie titles, and drop duplicate title s, if any 
indiceCosine = pd.Series(overview.index, index=overview['title']).drop_duplicates( ) 
# indiceID = pd.Series(overview.index, index=overview['id']).drop_duplicates() 
# indiceTitulo = overview[['id', 'title']] #id_para_titulo = indiceTitulo.set_index('id') 
# titulo_para_id = indiceTitulo.set_index('title')
# indiceTitulo.head() 
# id_para_titulo.head() 
# print(id_para_titulo.loc['137']['title']) 
# print(titulo_para_id.loc['Heat']['id']) 


title
Toy Story            0
Heat                 5
GoldenEye            9
Cutthroat Island    14
Casino              15
dtype: int64

### Aplicando SVD